In [1]:
import pandas as pd
import numpy as np

In [2]:
books = pd.read_excel(open('Beef Stew Book Club.xlsx', 'rb'), sheet_name='Books')l
latest_book = books.sort_values('Book Club Date').tail(1)['Book'].iloc[0]
books.sort_values('Book Club Date',ascending=False)

,Book,Book Club Date,Selector,Genre,Author,Pages,GoodReads,Amazon
4,Half a King,2024-04-20,Don,Fantasy,Joe Abercrombie,416,3.97,4.3
3,Tomorrow Tomorrow Tomorrow,2024-03-30,Gena,Coming of Age,Gabrielle Zevin,401,4.18,4.4
2,The Fury,2024-02-24,James,Mystery Thriller,Alex Michaelides,298,3.44,3.7
1,The Only One Left,2024-01-27,Gina,Mystery Thriller,Riley Sager,385,4.15,4.4
0,The Guest List,2023-12-03,Corey,Mystery Thriller,Lucy Foley,319,3.82,4.2


In [3]:
ratings = pd.read_excel(open('Beef Stew Book Club.xlsx', 'rb'), sheet_name='Rating')
ratings['Diff'] = abs(ratings['Rating'] - ratings['Pred'])
ratings = pd.merge(ratings,books, how='left', left_on=['Book'], right_on=['Book'])

In [4]:
ratings.groupby('Book')['Rating'].mean().sort_values(ascending=False)

Book
Half a King                   3.875000
The Fury                      3.708333
The Only One Left             3.541667
The Guest List                3.250000
Tomorrow Tomorrow Tomorrow    2.041667
Name: Rating, dtype: float64

In [5]:
ratings[ratings['Book']==latest_book]

,Reader,Book,Rating,Pred,Diff,Book Club Date,Selector,Genre,Author,Pages,GoodReads,Amazon
24,Corey,Half a King,4.00,2.19,1.81,2024-04-20,Don,Fantasy,Joe Abercrombie,416,3.97,4.3
25,Don,Half a King,4.50,2.24,2.26,2024-04-20,Don,Fantasy,Joe Abercrombie,416,3.97,4.3
26,Gena,Half a King,3.50,2.30,1.20,2024-04-20,Don,Fantasy,Joe Abercrombie,416,3.97,4.3
27,Gina,Half a King,4.25,2.32,1.93,2024-04-20,Don,Fantasy,Joe Abercrombie,416,3.97,4.3
28,Hannah,Half a King,3.25,1.86,1.39,2024-04-20,Don,Fantasy,Joe Abercrombie,416,3.97,4.3
29,James,Half a King,3.75,2.27,1.48,2024-04-20,Don,Fantasy,Joe Abercrombie,416,3.97,4.3


In [6]:
ratings[ratings['Book']==latest_book].Rating.mean()

3.875

In [7]:
#Genre Count
books.groupby('Genre')['Book'].count().sort_values(ascending=False)

Genre
Mystery Thriller    3
Coming of Age       1
Fantasy             1
Name: Book, dtype: int64

In [8]:
#Genre Average
round(ratings.groupby('Genre')['Rating'].mean().sort_values(ascending=False),2)

Genre
Fantasy             3.88
Mystery Thriller    3.50
Coming of Age       2.04
Name: Rating, dtype: float64

In [9]:
bookratings = ratings.groupby('Book')['Rating'].mean()

In [10]:
# Top Books
round(pd.DataFrame(bookratings).reset_index().sort_values('Rating',ascending=False).head(5),2).merge(books)

,Book,Rating,Book Club Date,Selector,Genre,Author,Pages,GoodReads,Amazon
0,Half a King,3.88,2024-04-20,Don,Fantasy,Joe Abercrombie,416,3.97,4.3
1,The Fury,3.71,2024-02-24,James,Mystery Thriller,Alex Michaelides,298,3.44,3.7
2,The Only One Left,3.54,2024-01-27,Gina,Mystery Thriller,Riley Sager,385,4.15,4.4
3,The Guest List,3.25,2023-12-03,Corey,Mystery Thriller,Lucy Foley,319,3.82,4.2
4,Tomorrow Tomorrow Tomorrow,2.04,2024-03-30,Gena,Coming of Age,Gabrielle Zevin,401,4.18,4.4


In [11]:
# Bottom Books
round(pd.DataFrame(bookratings).reset_index().sort_values('Rating',ascending=True).head(5),2).merge(books)

,Book,Rating,Book Club Date,Selector,Genre,Author,Pages,GoodReads,Amazon
0,Tomorrow Tomorrow Tomorrow,2.04,2024-03-30,Gena,Coming of Age,Gabrielle Zevin,401,4.18,4.4
1,The Guest List,3.25,2023-12-03,Corey,Mystery Thriller,Lucy Foley,319,3.82,4.2
2,The Only One Left,3.54,2024-01-27,Gina,Mystery Thriller,Riley Sager,385,4.15,4.4
3,The Fury,3.71,2024-02-24,James,Mystery Thriller,Alex Michaelides,298,3.44,3.7
4,Half a King,3.88,2024-04-20,Don,Fantasy,Joe Abercrombie,416,3.97,4.3


In [12]:
# Who likes books
likesbooks = pd.DataFrame(ratings.groupby('Reader')['Rating'].mean().sort_values(ascending=False).round(2))
likesbooks = likesbooks.rename(columns={'Rating': 'Avg_Rating'}) 
likesbooks['CurrentRank'] = likesbooks['Avg_Rating'].rank(ascending=False, method="min")

latestbook = ratings['Book Club Date'].max()

likesbooks_last = ratings[ratings['Book Club Date']<latestbook]
likesbooks_last = pd.DataFrame(likesbooks_last.groupby('Reader')['Rating'].mean().sort_values(ascending=False).round(2))
likesbooks_last['CurrentRank_Last'] = likesbooks_last['Rating'].rank(ascending=False, method="min")

likesbooks = pd.merge(likesbooks,likesbooks_last[['CurrentRank_Last']], how='left', left_index=True, right_index=True)

likesbooks['Rank_Change'] = np.where(likesbooks['CurrentRank'] < likesbooks['CurrentRank_Last'],'Up',np.NaN)
likesbooks['Rank_Change'] = np.where(likesbooks['CurrentRank'] == likesbooks['CurrentRank_Last'],'Same',likesbooks['Rank_Change'])
likesbooks['Rank_Change'] = np.where(likesbooks['CurrentRank'] > likesbooks['CurrentRank_Last'],'Down',likesbooks['Rank_Change'])



fav_books = ratings.copy()
fav_books['favbook'] = fav_books.groupby("Reader")["Rating"].rank(method="min", ascending=False)#['Reader'].rank(ascending=False, method="min")#ratings.sort_values('Rating',ascending=False).drop_duplicates(subset=['Reader'], keep='first')
fav_books = fav_books[fav_books['favbook']==1]
fav_books_count = fav_books.groupby(['Reader'])['Book'].count()
fav_books = fav_books.drop_duplicates(subset=['Reader'], keep='first')
fav_books = fav_books.rename(columns={'Rating': 'Fav_Rating'}) 

likesbooks = pd.merge(likesbooks,fav_books_count, how='left', left_on=['Reader'], right_on=['Reader'])
likesbooks = pd.merge(likesbooks,fav_books[['Reader','Book','Fav_Rating']], how='left', left_on=['Reader'], right_on=['Reader'])
likesbooks['Favorite_Book'] = np.where(likesbooks['Book_x'] == 1, likesbooks['Book_y'], likesbooks['Book_x'].astype(str)+' Books Tied')

likesbooks = likesbooks.drop(columns=['CurrentRank_Last','Book_x','Book_y'])
likesbooks

,Reader,Avg_Rating,CurrentRank,Rank_Change,Fav_Rating,Favorite_Book
0,Don,3.65,1.0,Same,4.50,Half a King
1,Gina,3.55,2.0,Same,4.25,2 Books Tied
2,Corey,3.30,3.0,Same,4.00,Half a King
3,James,3.20,4.0,Same,3.75,2 Books Tied
4,Gena,3.15,5.0,Down,3.50,2 Books Tied
5,Hannah,2.85,6.0,Same,3.75,The Only One Left


In [13]:
# graph
ratings_pivot = ratings.copy()
ratings_pivot['Rating_Rolling_Average'] = ratings_pivot.groupby('Reader')['Rating'].transform(lambda x: x.rolling(100, 1).mean())

ratings_pivot['Rating_Rolling_Average'] = ratings_pivot['Rating_Rolling_Average'] - 2.5
ratings_pivot = pd.pivot_table(ratings_pivot, values = 'Rating_Rolling_Average', index=['Book Club Date'], columns = 'Reader').reset_index()
ratings_pivot.columns.name = None
ratings_pivot.style.hide()

Book Club Date,Corey,Don,Gena,Gina,Hannah,James
2023-12-03 00:00:00,0.625000,1.250000,0.625000,1.250000,1.000000,1.125000
2024-01-27 00:00:00,0.750000,1.166667,0.666667,1.250000,1.083333,1.083333
2024-02-24 00:00:00,0.750000,1.500000,1.000000,1.750000,1.000000,1.250000
2024-03-30 00:00:00,0.625000,0.937500,0.562500,0.875000,0.250000,0.562500
2024-04-20 00:00:00,0.800000,1.150000,0.650000,1.050000,0.350000,0.700000


In [14]:
# Selects good books

selectsbooks = pd.DataFrame(ratings.groupby('Selector')['Rating'].mean().sort_values(ascending=False).round(2))
selectsbooks = selectsbooks.rename(columns={'Rating': 'Avg_Rating'}) 
selectsbooks['CurrentRank'] = selectsbooks['Avg_Rating'].rank(ascending=False, method="min")


latestbook = ratings['Book Club Date'].max()

selectsbooks_last = ratings[ratings['Book Club Date']<latestbook]
selectsbooks_last = pd.DataFrame(selectsbooks_last.groupby('Selector')['Rating'].mean().sort_values(ascending=False).round(2))
selectsbooks_last['CurrentRank_Last'] = selectsbooks_last['Rating'].rank(ascending=False, method="min")

selectsbooks = pd.merge(selectsbooks,selectsbooks_last[['CurrentRank_Last']], how='left', left_index=True, right_index=True)

selectsbooks['Rank_Change'] = np.where(selectsbooks['CurrentRank'] < selectsbooks['CurrentRank_Last'],'Up',np.NaN)
selectsbooks['Rank_Change'] = np.where(selectsbooks['CurrentRank'] == selectsbooks['CurrentRank_Last'],'Same',selectsbooks['Rank_Change'])
selectsbooks['Rank_Change'] = np.where(selectsbooks['CurrentRank'] > selectsbooks['CurrentRank_Last'],'Down',selectsbooks['Rank_Change'])



best_books = ratings.copy()
best_books = best_books[['Book','Book Club Date','Selector']].drop_duplicates()

best_books_count = pd.DataFrame(best_books.groupby(['Selector'])['Book'].count())
best_books_count = best_books_count.rename(columns={'Book': 'Books Selected'}).reset_index()

best_books = pd.merge(best_books,bookratings, how='left', left_on=['Book'], right_on=['Book'])
best_books['bestbook'] = best_books.groupby("Selector")["Rating"].rank(method="min", ascending=False)#['Reader'].rank(ascending=False, method="min")#ratings.sort_values('Rating',ascending=False).drop_duplicates(subset=['Reader'], keep='first')
best_books = best_books[best_books['bestbook']==1]
best_books_count = best_books_count.groupby(['Selector'])['Books Selected'].count()
best_books = best_books.drop_duplicates(subset=['Selector'], keep='first')
best_books = best_books.rename(columns={'Rating': 'Best_Rating'}) 

selectsbooks = pd.merge(selectsbooks,best_books_count, how='left', left_on=['Selector'], right_on=['Selector'])
selectsbooks = pd.merge(selectsbooks,best_books[['Selector','Book','Best_Rating']], how='left', left_on=['Selector'], right_on=['Selector'])
selectsbooks['Best_Book'] = np.where(selectsbooks['Books Selected'] == 1, selectsbooks['Book'], selectsbooks['Books Selected'].astype(str)+' Books Tied')

selectsbooks = selectsbooks.drop(columns=['CurrentRank_Last','Books Selected','Book'])
selectsbooks

,Selector,Avg_Rating,CurrentRank,Rank_Change,Best_Rating,Best_Book
0,Don,3.88,1.0,nan,3.875000,Half a King
1,James,3.71,2.0,Down,3.708333,The Fury
2,Gina,3.54,3.0,Down,3.541667,The Only One Left
3,Corey,3.25,4.0,Down,3.250000,The Guest List
4,Gena,2.04,5.0,Down,2.041667,Tomorrow Tomorrow Tomorrow


In [15]:
train = ratings[ratings['Book Club Date']<'2024-3-20'] #Date of next book club
train = train[['Reader','Rating','Selector','Genre','Pages','GoodReads','Amazon']]
train

,Reader,Rating,Selector,Genre,Pages,GoodReads,Amazon
0,Corey,3.25,James,Mystery Thriller,298,3.44,3.7
1,Don,4.00,James,Mystery Thriller,298,3.44,3.7
2,Gena,3.50,James,Mystery Thriller,298,3.44,3.7
3,Gina,4.25,James,Mystery Thriller,298,3.44,3.7
4,Hannah,3.50,James,Mystery Thriller,298,3.44,3.7
5,James,3.75,James,Mystery Thriller,298,3.44,3.7
6,Corey,3.00,Corey,Mystery Thriller,319,3.82,4.2
7,Don,3.50,Corey,Mystery Thriller,319,3.82,4.2
8,Gena,2.75,Corey,Mystery Thriller,319,3.82,4.2
9,Gina,3.25,Corey,Mystery Thriller,319,3.82,4.2


In [16]:
from pycaret.regression import *
s = setup(train, target = 'Rating', session_id = 123)

,Description,Value
0,Session id,123
1,Target,Rating
2,Target type,Regression
3,Original data shape,"(18, 7)"
4,Transformed data shape,"(18, 14)"
5,Transformed train set shape,"(12, 14)"
6,Transformed test set shape,"(6, 14)"
7,Numeric features,3
8,Categorical features,3
9,Preprocess,True


In [17]:
# best = compare_models()
lr = create_model('lr', fold = 3)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.2031,0.1221,0.3494,-1.8409,0.0764,0.0625
1,0.4375,0.2969,0.5449,-0.2881,0.1156,0.1088
2,0.2531,0.0825,0.2872,-6.0367,0.0637,0.0720
Mean,0.2979,0.1671,0.3938,-2.7219,0.0852,0.0811
Std,0.1008,0.0932,0.1098,2.4281,0.0221,0.0200


In [18]:
evaluate_model(lr)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [19]:
# Get predictions for next book
preds = books[books['Book']==latest_book]
preds = preds.loc[preds.index.repeat(6)].reset_index(drop=True)
preds = pd.merge(preds,pd.DataFrame(ratings.Reader.unique()), how='outer', left_index=True, right_index=True)
preds = preds.rename(columns={0: 'Reader'}) 
preds = preds[['Reader','Selector','Genre','Pages','GoodReads','Amazon']]

predictions = round(predict_model(lr, data=preds).merge(ratings[ratings['Book']==latest_book][['Reader','Rating']],left_on=['Reader'],right_on=['Reader']),2)
predictions['Diff'] = predictions['Rating'] - predictions['prediction_label']
predictions

,Reader,Selector,Genre,Pages,GoodReads,Amazon,prediction_label,Rating,Diff
0,Corey,Don,Fantasy,416,3.97,4.3,3.24,4.00,0.76
1,Don,Don,Fantasy,416,3.97,4.3,3.57,4.50,0.93
2,Gena,Don,Fantasy,416,3.97,4.3,3.18,3.50,0.32
3,Gina,Don,Fantasy,416,3.97,4.3,3.74,4.25,0.51
4,Hannah,Don,Fantasy,416,3.97,4.3,3.77,3.25,-0.52
5,James,Don,Fantasy,416,3.97,4.3,3.45,3.75,0.30


In [20]:
pred_diffs = pd.DataFrame(ratings.groupby('Book')['Diff'].mean().dropna()).merge(books, how='left', left_on=['Book'], right_on=['Book']).sort_values('Book Club Date',ascending=True)[['Book Club Date','Diff']]
# pred_diffs
pred_diffs.columns.name = None
pred_diffs.style.hide()

Book Club Date,Diff
2024-01-27 00:00:00,0.250000
2024-02-24 00:00:00,0.240000
2024-03-30 00:00:00,1.623333
2024-04-20 00:00:00,1.678333
